In [1]:
demo_params = {
    "num_units": "32,16",
    "activation_func": "Sigmoid",
    "batch_size": 16,
    "num_epochs": 50,
    "learning_rate": 1e-2,
    "momentum": 0.9,
    "test_ratio": 0.2,
    
    "input_data": "./data/Parkinsons.data",
    "loss_func": "square_loss",
    "l2_norm": 1e-3
}
demo_params

{'num_units': '32,16',
 'activation_func': 'Sigmoid',
 'batch_size': 16,
 'num_epochs': 50,
 'learning_rate': 0.01,
 'momentum': 0.9,
 'test_ratio': 0.2,
 'input_data': './data/Parkinsons.data',
 'loss_func': 'square_loss',
 'l2_norm': 0.001}

In [17]:
"A".lower()

'a'

In [2]:
from tinytorch import log_binary_loss, Tensor

y_pred = Tensor([[0], [0]])
y = Tensor([[1], [0]])
log_binary_loss(y_pred, y).values

array([[0.69314718]])

In [15]:
import numpy as np
from tinytorch import Tensor, MLP, Parameter, MomentumOptimizer, Linear, Sigmoid, square_loss, log_binary_loss
from dataset import DataLoader

def train_func(**params):
    num_units = [int(x) for x in params["num_units"].split(',')]
    activation_func = params["activation_func"]
    batch_size = int(params["batch_size"])
    num_epochs = int(params["num_epochs"])
    learning_rate = float(params["learning_rate"])
    momentum = float(params["momentum"])
    l2_norm_coef = float(params["l2_norm"])
    
    test_ratio = float(params["test_ratio"])
    input_data = np.genfromtxt(params["input_data"], delimiter=',')[1:]
    
    train_data = input_data[: int(input_data.shape[0] * (1-test_ratio))]
    test_data = input_data[int(input_data.shape[0] * (1-test_ratio)):]
    
    
    log_step = int(params.get("log_step", 1))
    
    loss_func = None
    
    if params["loss_func"] == "square_loss":
        loss_func = square_loss
    elif params["loss_func"] == "log_binary_loss":
        loss_func = log_binary_loss
    assert train_data.shape[-1] == test_data.shape[-1]
    
    print(train_data.shape, test_data.shape)
    
    Parameter.clear()
    mlp = MLP(train_data.shape[-1]-1, 1, num_units, "Sigmoid", use_bias=False)
    output_layer = Linear(num_units[-1], 1, use_bias=False)
    
    optimizer = MomentumOptimizer(Parameter.param_list, learning_rate, momentum)

    
    for epoch in range(num_epochs):
        
        for batch_idx, (batch_X, batch_y) in enumerate(DataLoader(train_data, shuffle=True, batch_size=16)):
            # print(batch_idx)
            input_x = Tensor(batch_X, requires_grad=False)
            output_y = Tensor(np.expand_dims(batch_y,-1), requires_grad=False)
            # print(input_x.values)
            y_pred = mlp.forward(input_x)
            loss = loss_func(y_pred,  output_y)

            l2_loss = Tensor(np.zeros((1,1)))
            for param in optimizer.params:
                l2_loss = l2_loss + param.l2_norm(l2_norm_coef)

            total_loss = loss + l2_loss
            total_loss.backward()
            # l2_loss.backward()
            optimizer.step()
        # break
            
            
        # if(epoch % 10 == 0):
        if epoch % log_step == 0:
            print("Epoch %d" % epoch)
            for batch_idx, (batch_X, batch_y) in enumerate(DataLoader(train_data, shuffle=True, batch_size=train_data.shape[0])):
                input_x = Tensor(batch_X, requires_grad=False)
                output_y = Tensor(np.expand_dims(batch_y,-1), requires_grad=False)

                y_pred = mlp.forward(input_x)
                loss = loss_func(y_pred,  output_y)

                l2_loss = Tensor(np.zeros((1,1)))
                for param in optimizer.params:
                    l2_loss = l2_loss + param.l2_norm(l2_norm_coef)

                # total_loss = loss + l2_loss
                print("train_loss:", loss.values)
                # print("train_l2_loss:", l2_loss.values)
            
            # print("Epoch %d" % epoch)
            for batch_idx, (batch_X, batch_y) in enumerate(DataLoader(test_data, shuffle=True, batch_size=test_data.shape[0])):
                input_x = Tensor(batch_X, requires_grad=False)
                output_y = Tensor(np.expand_dims(batch_y,-1), requires_grad=False)

                y_pred = mlp.forward(input_x)
                loss = loss_func(y_pred,  output_y)

                l2_loss = Tensor(np.zeros((1,1)))
                for param in optimizer.params:
                    l2_loss = l2_loss + param.l2_norm(l2_norm_coef)

                # loss = loss + l2_loss
                print("test_loss:", loss.values)
                # print("test_l2_loss:", l2_loss.values)
#             if(batch_idx % 10) == 0:
#                 print(epoch, batch_idx, loss.values)
    

In [ ]:
mlp.

In [7]:
input_x.shape, output_y.shape

((16, 30), (16,))

In [66]:
type(mlp.activative_func) == str

True

In [5]:
1/ np.log([1.1,2,3])

array([10.49205869,  1.44269504,  0.91023923])

In [17]:
demo_params = {
    "num_units": "16,16",
    "activation_func": "Sigmoid",
    "batch_size": 32,
    "num_epochs": 200,
    "learning_rate": 1e-2,
    "momentum": 0.9,
    "test_ratio": 0.2,
    
    "input_data": "./data/Parkinsons.data",
    "loss_func": "square_loss",
    # "l2_norm": 0.1
    "l2_norm": 5e-3,
    "log_step":20
}
demo_params

train_func(**demo_params)

(4700, 20) (1175, 20)
Epoch 0
train_loss: [[0.76291621]]
test_loss: [[2.35776374]]
Epoch 20
train_loss: [[0.52012711]]
test_loss: [[1.75037505]]
Epoch 40
train_loss: [[0.47606547]]
test_loss: [[1.78521449]]
Epoch 60
train_loss: [[0.44665647]]
test_loss: [[1.80038001]]
Epoch 80
train_loss: [[0.43584233]]
test_loss: [[1.961267]]
Epoch 100
train_loss: [[0.41709626]]
test_loss: [[1.83921424]]
Epoch 120
train_loss: [[0.41705985]]
test_loss: [[1.97418836]]
Epoch 140
train_loss: [[0.4073712]]
test_loss: [[1.84754531]]
Epoch 160
train_loss: [[0.40582123]]
test_loss: [[1.82282351]]
Epoch 180
train_loss: [[0.42029779]]
test_loss: [[1.52472262]]


In [18]:
demo_params = {
    "num_units": "16,16",
    "activation_func": "Tanh",
    "batch_size": 32,
    "num_epochs": 100,
    "learning_rate": 1e-2,
    "momentum": 0.9,
    "test_ratio": 0.2,
    
    "input_data": "./data/BreastCancer.data",
    "loss_func": "log_binary_loss",
    # "l2_norm": 0.1
    "l2_norm": 5e-3,
    "log_step": 10
}
demo_params

train_func(**demo_params)

(455, 31) (114, 31)
Epoch 0
train_loss: [[1.12311733]]
test_loss: [[0.84992098]]
Epoch 10
train_loss: [[0.57138392]]
test_loss: [[0.52123852]]
Epoch 20
train_loss: [[0.37228178]]
test_loss: [[0.36742945]]
Epoch 30
train_loss: [[0.28607235]]
test_loss: [[0.29547223]]
Epoch 40
train_loss: [[0.2392933]]
test_loss: [[0.25225273]]
Epoch 50
train_loss: [[0.20896204]]
test_loss: [[0.22196062]]
Epoch 60
train_loss: [[0.18749544]]
test_loss: [[0.20122427]]
Epoch 70
train_loss: [[0.1713417]]
test_loss: [[0.18390648]]
Epoch 80
train_loss: [[0.1586874]]
test_loss: [[0.1716861]]
Epoch 90
train_loss: [[0.14871547]]
test_loss: [[0.16137102]]


In [1]:
import numpy as np

from dataset import DataLoader

input_data_loader = DataLoader(np.genfromtxt("./data/BreastCancer.data", delimiter=',')[1:],  shuffle=True, batch_size=16)

[7 6 5 9 8 2 3 4 1 0]


In [4]:
X.shape

(16, 30)

In [2]:
for i, (X, y) in enumerate(input_data_loader):
    print(i, X.shape, y.shape, X[0], y)
    
    if i == 5:
        break
    # break

0 (16, 30) (16,) [1.320e+01 1.743e+01 8.413e+01 5.416e+02 7.215e-02 4.524e-02 4.336e-02
 1.105e-02 1.487e-01 5.635e-02 1.630e-01 1.601e+00 8.730e-01 1.356e+01
 6.261e-03 1.569e-02 3.079e-02 5.383e-03 1.962e-02 2.250e-03 1.394e+01
 2.782e+01 8.828e+01 6.020e+02 1.101e-01 1.508e-01 2.298e-01 4.970e-02
 2.767e-01 7.198e-02] [0. 1. 0. 0. 1. 0. 1. 0. 1. 0. 1. 0. 1. 1. 0. 1.]
1 (16, 30) (16,) [1.115e+01 1.308e+01 7.087e+01 3.819e+02 9.754e-02 5.113e-02 1.982e-02
 1.786e-02 1.830e-01 6.105e-02 2.251e-01 7.815e-01 1.429e+00 1.548e+01
 9.019e-03 8.985e-03 1.196e-02 8.232e-03 2.388e-02 1.619e-03 1.199e+01
 1.630e+01 7.625e+01 4.408e+02 1.341e-01 8.971e-02 7.116e-02 5.506e-02
 2.859e-01 6.772e-02] [0. 0. 1. 0. 0. 0. 0. 1. 1. 1. 1. 0. 0. 0. 0. 1.]
2 (16, 30) (16,) [1.678e+01 1.880e+01 1.093e+02 8.863e+02 8.865e-02 9.182e-02 8.422e-02
 6.576e-02 1.893e-01 5.534e-02 5.990e-01 1.391e+00 4.129e+00 6.734e+01
 6.123e-03 2.470e-02 2.626e-02 1.604e-02 2.091e-02 3.493e-03 2.005e+01
 2.630e+01 1.307e+02 1.2

In [5]:
x

array([[1.877e+01, 2.143e+01, 1.229e+02, ..., 3.679e-01, 9.870e-02,
        1.000e+00],
       [1.175e+01, 1.756e+01, 7.589e+01, ..., 2.478e-01, 7.757e-02,
        0.000e+00],
       [1.613e+01, 1.788e+01, 1.070e+02, ..., 4.270e-01, 1.233e-01,
        1.000e+00],
       ...,
       [1.340e+01, 1.695e+01, 8.548e+01, ..., 2.741e-01, 7.582e-02,
        0.000e+00],
       [1.402e+01, 1.566e+01, 8.959e+01, ..., 2.136e-01, 6.710e-02,
        0.000e+00],
       [1.550e+01, 2.108e+01, 1.029e+02, ..., 3.003e-01, 1.048e-01,
        1.000e+00]])

In [4]:
input_data[1:]

NameError: name 'input_data' is not defined